# **Computing $w(\theta)$ for a given cosmology and a set of redshift distributions**

This notebook provides a step-by-step guide to calculate the theoretical angular correlation function \( w(\theta) \) based on a given cosmology and redshift distributions.

---

1. **Full integrals**:  
   The code computes $w(\theta)$ without using simplifications like the Limber or flat-sky approximations, ensuring accurate results.

2. **Parallel processing**:  
   By default, the code uses **100 CPUs** to speed up calculations. Make sure to run it on an interactive node.

3. **Optional no-wiggle template**:  
   You can generate a template without BAO wiggles to help evaluate the significance of the BAO detection.

---


In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
import numpy as np
from cosmoprimo import Cosmology
import argparse
from utils_template import PowerSpectrumMultipoles, CorrelationFunctionMultipoles, WThetaCalculator

# 1. Arguments
class Args:
    def __init__(self):
        self.cosmology_template = "planck"
        # self.nz_flag = "fid"
        self.nz_flag = "clusteringz"
        self.include_wiggles = "n"
args = Args()
args.include_wiggles = '' if args.include_wiggles == 'y' else '_nowiggles'

# 2. Numerical resolution of the calculation. The larger these numbers are, the more accurate the template will be
Nz, Nk, Nmu, Nr, Ntheta = 10**3, 2*10**5, 5*10**4, 5*10**4, 10**3 # these are the settings I used for DES Y6 BAO
# Nz, Nk, Nmu, Nr, Ntheta = 10**3, 10**4, 10**4, 10**4, 10**2

# 3. Calculation of pk_ell
pk_calculator = PowerSpectrumMultipoles(
    include_wiggles=args.include_wiggles,
    nz_flag=args.nz_flag,
    cosmology_template=args.cosmology_template,
    Nk=Nk,
    Nmu=Nmu,
    n_cpu=128
)
for bin_z in range(pk_calculator.nbins):
    pk_calculator.compute_pk_ell(bin_z)

# 4. Calculation of xi_ell
xi_calculator = CorrelationFunctionMultipoles(
    power_spectrum_multipoles=pk_calculator,
    Nr=Nr
)
for bin_z in range(pk_calculator.nbins):
    xi_calculator.compute_xi_ell(bin_z)

# 5. Calculation of w(theta)
wtheta_calculator = WThetaCalculator(
    correlation_function_multipoles=xi_calculator,
    Nz=Nz,
    Ntheta=Ntheta
)
for bin_z in range(pk_calculator.nbins):
    wtheta_calculator.compute_wtheta(bin_z)


Using clusteringz n(z), which has 4 redshift bins
Saving output to: wtheta_template_nowiggles/nz_clusteringz/wtheta_planck
Initialized cosmology: planck
0 - Computing Pk_ell...
0 - Pk_ell computed!
1 - Computing Pk_ell...
1 - Pk_ell computed!
2 - Computing Pk_ell...
2 - Pk_ell computed!
3 - Computing Pk_ell...
3 - Pk_ell computed!
0 - Computing xi_ell...
0 - Loading precomputed Pk_ell data...
0 - xi_ell computed!
1 - Computing xi_ell...
1 - Loading precomputed Pk_ell data...
1 - xi_ell computed!
2 - Computing xi_ell...
2 - Loading precomputed Pk_ell data...
2 - xi_ell computed!
3 - Computing xi_ell...
3 - Loading precomputed Pk_ell data...
3 - xi_ell computed!
0 - Computing w(theta)...
0 - Loading precomputed xi_ell data...
0 - w(theta) computed!
1 - Computing w(theta)...
1 - Loading precomputed xi_ell data...
1 - w(theta) computed!
2 - Computing w(theta)...
2 - Loading precomputed xi_ell data...
2 - w(theta) computed!
3 - Computing w(theta)...
3 - Loading precomputed xi_ell data...
3 